In [31]:
import gymnasium as gym
from es_grad_im import Actor
import argparse
import torch

In [32]:
class Args(argparse.Namespace):
    actor_lr=0.001
    critic_lr=0.001
    batch_size=100
    discount=0.99
    reward_scale=1.0
    tau=0.005
    layer_norm=False

    use_td3="use_td3"
    policy_noise=0.2
    noise_clip=0.5
    policy_freq=2

args=Args()


In [36]:
env = gym.make("HalfCheetah-v4", render_mode="human")
env = gym.wrappers.RecordEpisodeStatistics(env)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = int(env.action_space.high[0])
actor = Actor(state_dim, action_dim, max_action, args)
actor.load_model('results/HalfCheetah-v4-run8', 'actor')

observation, info = env.reset()

for _ in range(1000):
    state = torch.FloatTensor(observation.reshape(-1))
    action = actor(state).cpu().data.numpy().flatten()

    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()